# BigQuery APIs

Google Cloud Datalab provides an integrated environment for working with Google BigQuery for both adhoc, exploratory work as well as pipeline development. This notebook introduces some of the APIs that Cloud Datalab provides for working with BigQuery.

You've already seen the use of `%%bq` in the [Hello BigQuery](Hello BigQuery.ipynb) notebook, and various commands in the [BigQuery Commands](BigQuery Commands.ipynb) notebook. These BigQuery commands are built using the same BigQuery APIs that are available for your own use.

## Importing the API

The Cloud Datalab APIs are provided in the `datalab` Python library, and the BigQuery functionality is contained within the `google.datalab.bigquery` module.

In [1]:
import google.datalab.bigquery as bq

# Querying Data

The most important BigQuery-related API is the one that allows you to execute a SQL query. The `google.datalab.bigquery.Query` class provides that functionality.

In [2]:
# Create and run a SQL query
bq.Query('SELECT * FROM `cloud-datalab-samples.httplogs.logs_20140615` LIMIT 3').execute().result()

timestamp,latency,status,method,endpoint
2014-06-15 07:00:00.536486,48,200,GET,Interact3
2014-06-15 07:00:00.003772,122,200,GET,Interact3
2014-06-15 07:00:00.428897,144,200,GET,Interact3


## SQL

Let's run the query created above with caching turned off, so we're sure to be able to retrieve metadata, such as bytes processed from resulting query job.

For this, we'll need to use a `QueryOutput` object.

In [3]:
output_options = bq.QueryOutput.table(use_cache=False)
result = bq.Query('SELECT * FROM `cloud-datalab-samples.httplogs.logs_20140615` LIMIT 3').execute(output_options=output_options).result()
result

timestamp,latency,status,method,endpoint
2014-06-15 07:00:00.536486,48,200,GET,Interact3
2014-06-15 07:00:00.003772,122,200,GET,Interact3
2014-06-15 07:00:00.428897,144,200,GET,Interact3


The result object is a `QueryResultsTable` class, and can be enumerated in the same manner a regular Python list, in addition to retrieving metadata about the result.

In [4]:
# Inspecting the result, and the associated job
print result.sql
print str(result.length) + ' rows'
print str(result.job.bytes_processed) + ' bytes processed'

SELECT * FROM `cloud-datalab-samples.httplogs.logs_20140615` LIMIT 3
3 rows
24152138 bytes processed


In [5]:
# Inspect the programmatic representation.
# Converting the QueryResultsTable to a vanilla list enables viewing the literal data,
# as well as side-stepping the HTML rendering seen above.
list(result)

[{u'endpoint': 'Interact3',
  u'latency': 48,
  u'method': 'GET',
  u'status': 200,
  u'timestamp': datetime.datetime(2014, 6, 15, 7, 0, 0, 536486)},
 {u'endpoint': 'Interact3',
  u'latency': 122,
  u'method': 'GET',
  u'status': 200,
  u'timestamp': datetime.datetime(2014, 6, 15, 7, 0, 0, 3772)},
 {u'endpoint': 'Interact3',
  u'latency': 144,
  u'method': 'GET',
  u'status': 200,
  u'timestamp': datetime.datetime(2014, 6, 15, 7, 0, 0, 428897)}]

# Datasets and Tables

In addition to executing queries, BigQuery objects like Datasets, Tables and their Schemas can be accessed programmatically as well.

## Listing Resources

In [6]:
from google.datalab import Context
datasets = bq.Datasets(context = Context('cloud-datalab-samples', None))
for ds in datasets:
  print ds.name

DatasetName(project_id=u'cloud-datalab-samples', dataset_id=u'appenginelogs')
DatasetName(project_id=u'cloud-datalab-samples', dataset_id=u'carprices')
DatasetName(project_id=u'cloud-datalab-samples', dataset_id=u'httplogs')


In [7]:
sample_dataset = list(datasets)[1]
tables = sample_dataset.tables()
for table in tables:
  print '%s (%d rows - %d bytes)' % (table.name.table_id, table.metadata.rows, table.metadata.size)

testing (100 rows - 4586 bytes)
training (417 rows - 19086 bytes)


In [8]:
table = bq.Table('cloud-datalab-samples.httplogs.logs_20140615')
fields = map(lambda tsf: tsf.name, table.schema)
fields

[u'timestamp', u'latency', u'status', u'method', u'endpoint']

## Creating Resources

In [9]:
# Create a new dataset (this will be deleted later in the notebook)
sample_dataset = bq.Dataset('apisample')
sample_dataset.create(friendly_name = 'Sample Dataset', description = 'Created from Sample Notebook')
sample_dataset.exists()

True

In [10]:
# To create a table, we also need to create a schema.
# Its easiest to create a schema from some existing data, so this
# example demonstrates using an example object
sample_row = {
  'name': 'string value',
  'value': 0,
  'flag': True
}
sample_schema = bq.Schema.from_data([sample_row])

sample_table = bq.Table("apisample.sample_table").create(schema = sample_schema, overwrite = True)

You can run the cell, below, to see the contents of the new dataset:

In [ ]:
list(sample_dataset.tables())

## Deleting Resources

In [12]:
# Clear out sample resources
sample_dataset.delete(delete_contents = True)

# Looking Ahead

This notebook covered a small subset of the APIs. Subsequent notebooks cover additional capabilities, such as importing and exporting data into and from BigQuery tables.